In [1]:
from google.colab import drive
drive.mount('/content/drive')
dir_path = '/content/drive/My Drive/Colab Notebooks'
import os
os.chdir(dir_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import gensim
import re
import torch
import numpy as np

w2v_path = 'GoogleNews-vectors-negative300.bin (1).gz'
data_path = 'report8/data70/'

w2v_model = gensim.models.KeyedVectors.load_word2vec_format(w2v_path , binary=True)

category2num = {"b": 0, "t": 1, "e": 2, "m": 3}

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


70

In [32]:
# タイトルを受け取り，単語ベクトルの平均を返す関数
def get_feature(title):
    word_list = title.split(' ')#todo タイトルをスペースで分割
    vec_list = []
    for word in word_list:
        try:
            vec = w2v_model[word]#todo wordを意味するベクトルを取得
        except KeyError:
            vec = np.zeros(300)#todo すべての要素が0のベクトルを代入
        vec_list.append(vec)    
    vec_np = np.array(vec_list) # numpyのarrayに変換
    feature = np.sum(vec_np, axis=0)/len(vec_np)# 平均ベクトルを計算
    return feature


def get_data(fname):
    label_list = []
    feature_list = []
    with open(fname, encoding="utf8") as f:
        for line in f:
            if not line:
                continue
            data = line.split('\t')
            title = data[1]#todo dataからタイトルを取り出す
            feature = get_feature(title) 
            feature_list.append(feature)
            label = category2num[data[0]]
            label_list.append(label)

    features = torch.tensor(feature_list)#todo feature_listをtensorに変換
    labels = torch.tensor(label_list)#todo label_listをtensorに変換
    return features, labels


train_x, train_y = get_data(data_path + "train.txt")
valid_x, valid_y = get_data(data_path + "valid.txt")
test_x, test_y = get_data(data_path + "test.txt")

# 保存
torch.save(train_x, data_path + "train_x.pt")
torch.save(train_y, data_path + "train_y.pt")
torch.save(valid_x, data_path + "valid_x.pt")
torch.save(valid_y, data_path + "valid_y.pt")
torch.save(test_x, data_path + "test_x.pt")
torch.save(test_y, data_path + "test_y.pt")

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import time

# 単層ニューラルネットワークを定義
class SingleLayerNN(nn.Module):
    def __init__(self, embedding_dim, num_labels):
        super(SingleLayerNN, self).__init__()
        self.linear = nn.Linear(embedding_dim, num_labels) #todo linear層を追加 4次元の結果が帰ってくる
    
    def forward(self, x):
        h1 = self.linear(x)#todo xをlinear層に入力
        softmax = nn.Softmax()
        return softmax(h1) #todo h1をsoftmaxしたものを返す

class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)  #todo データサイズを返す

    def __getitem__(self, idx):
        out_x = self.x[idx]#todo idx番目のxの要素を返す
        out_y = self.y[idx]#todo idx番目のyの要素を返す
        return out_x, out_y
        
torch.manual_seed(1)

embedding_dim = 300
num_labels = 4
NUM_EPOCH = 100

train_x, train_y = torch.load("./report8/data70/train_x.pt"), torch.load("./report8/data70/train_y.pt")
valid_x, valid_y = torch.load("./report8/data70/valid_x.pt"), torch.load("./report8/data70/valid_y.pt")


# データとラベルを1つにまとめる
dataset = MyDataset(train_x, train_y)

loss_fn = nn.CrossEntropyLoss()#todo 損失関数としてCrossEntropyLossを用いる


def train(model, train_loader):
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        pred = model(data.type(torch.FloatTensor))#todo dataをmodelに入力
        loss = loss_fn(pred, target)#todo predとtargetと損失関数からロスを計算
        loss.backward() # 逆誤差伝搬を実施
        optimizer.step() # パラメータを更新


def evaluation(model, data, target):
    model.eval()
    with torch.no_grad():
        pred = model.forward(data.type(torch.FloatTensor))#todo dataをmodelに入力
        loss = loss_fn(pred, target)#todo predとtargetと損失関数からロスを計算
        pred_labels = torch.argmax(pred, axis=1)#todo 推定したラベルを代入　どの次元が一番確率が高いか　max
        acc = (pred_labels==target).sum().item()/len(pred_labels)#todo pred_labelsとtargetから正解率を計算
    return loss.item(), acc


batch_size_list = [2**x for x in range(11)]#todo バッチサイズの値のリスト．1,2,4,8,…が入る


for batch_size in batch_size_list:
    model = SingleLayerNN(embedding_dim, num_labels)
    optimizer = optim.SGD(model.parameters(), lr=0.1)

    # ミニバッチを扱うためのデータローダを作成
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(NUM_EPOCH):
        start_time = time.time() # 学習開始時の時間を取得
        train(model, train_loader)
        took_time = time.time() - start_time #todo 学習にかかった時間を代入
        #print(valid_y)
        valid_loss, valid_acc = evaluation(model, valid_x, valid_y)

        if epoch % 10 == 0:
            print(f"batch size:{batch_size}\tepoch: {epoch}")
            print(f"train time: {took_time}")
            print(f"<valid>\tloss: {valid_loss}\tacc: {valid_acc}")
    
    print("\n================\n")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


batch size:1	epoch: 0
train time: 5.432270288467407
<valid>	loss: 0.9587206244468689	acc: 0.800599700149925
batch size:1	epoch: 10
train time: 5.3603832721710205
<valid>	loss: 0.9397221803665161	acc: 0.8013493253373314
batch size:1	epoch: 20
train time: 5.406141519546509
<valid>	loss: 0.9378631114959717	acc: 0.8020989505247377
batch size:1	epoch: 30
train time: 5.465374946594238
<valid>	loss: 0.936996340751648	acc: 0.8035982008995503
batch size:1	epoch: 40
train time: 5.5977911949157715
<valid>	loss: 0.9364750981330872	acc: 0.802848575712144
batch size:1	epoch: 50
train time: 5.488953113555908
<valid>	loss: 0.9360454082489014	acc: 0.8020989505247377
batch size:1	epoch: 60
train time: 5.544880390167236
<valid>	loss: 0.8930370211601257	acc: 0.8538230884557722
batch size:1	epoch: 70
train time: 5.4050493240356445
<valid>	loss: 0.8867973685264587	acc: 0.8590704647676162
batch size:1	epoch: 80
train time: 5.4647557735443115
<valid>	loss: 0.8853724002838135	acc: 0.8598200899550225
batch size

バッチ１のとき後半で値が飛ぶところがあったが、それ以外は順調に学習できた。